In [1]:
from keras.datasets import mnist 
import matplotlib.pyplot as plt 
from keras.optimizers import Adam 
from keras import utils 
import numpy as np 
import cv2
import matplotlib.pyplot as plt 
from tensorflow.keras.models import load_model 

dataset = mnist
(x_train, y_train), (x_test, y_test) = dataset.load_data() #translate the data to image 

x_test = x_test.reshape(x_test.shape[0], -1).astype("float64")

#normalization 
x_test /= 255

x_test = x_test.reshape(-1, 28, 28, 1)

alexnet = load_model('Model/alexnet_mnist.h5')#load lenet5 model trained with mnist dataset 
alexnet.compile(optimizer=Adam(),loss='sparse_categorical_crossentropy',metrics=['accuracy']) 

In [7]:
beta = 0.10791
sample(x_test,y_test,beta)

Beta is reduced by 0.0001, the new beta is 0.10781
Beta is reduced by 0.0001, the new beta is 0.10771
Beta is reduced by 0.0001, the new beta is 0.10761
Beta is reduced by 0.0001, the new beta is 0.10751
Beta is reduced by 0.0001, the new beta is 0.10740999999999999
Beta is reduced by 0.0001, the new beta is 0.10730999999999999
Beta is reduced by 0.0001, the new beta is 0.10720999999999999
Beta is reduced by 0.0001, the new beta is 0.10710999999999998
Beta is reduced by 0.0001, the new beta is 0.10700999999999998
Beta is reduced by 0.0001, the new beta is 0.10690999999999998
Beta is reduced by 0.0001, the new beta is 0.10680999999999997
Beta is reduced by 0.0001, the new beta is 0.10670999999999997
Beta is reduced by 0.0001, the new beta is 0.10660999999999997
Beta is reduced by 0.0001, the new beta is 0.10650999999999997
Beta is reduced by 0.0001, the new beta is 0.10640999999999996
Beta is reduced by 0.0001, the new beta is 0.10630999999999996
Beta is reduced by 0.0001, the new beta 

0.09590999999999966

In [6]:
def sample(x_test,y_test,beta):
    mean = np.zeros(2)  
    cov = np.eye(2)
    
    for i in range(x_test.shape[0]):
        count = 0
        while True:
            p = np.random.multivariate_normal(mean, cov,(28,28)) #sample formula
            p = p.reshape(-1,28,28,1)
            neighbour = p[0] + x_test[i]*255
            neighbour /= 255
            distance=np.linalg.norm(p[0]) 
            distance /= 255
            #print(distance)
            if (distance < beta or distance == beta):
                output = lenet5.predict(neighbour.reshape(-1, 28, 28, 1),batch_size=128)[0]
                proba = softmax(output)
                pred = np.argmax(proba)
                label = y_test[i]
                
                if(pred == label): 
                    count+=1

                else:  #if there are some iamges inside beta cannot have the same label, beta will be decreased
                    beta = beta-0.0001
                    print("Beta is reduced by 0.0001, the new beta is",beta)  
                    i-=1
                    break
                    
            if(count<600):  
                continue
                
            if(count==600):  
                #local robustness formula satisfied
                break
                
    print("global robustness formula satisfied")    
    print("max safe distance:",beta)
    return beta

def softmax(x):
    e_x = np.exp(x-np.max(x))
    return e_x / e_x.sum()